# Buffer

* Buffer is a python class in the sc3nb package to interface with Buffers on the SuperCollider3 server.
* The constructor is invoked by the method Buffer(), as attribute of a booted SC instance
        buf = sc.Buffer()
  to allocate a buffer on the sound server.
* The buffer instance is returned so that subsequent buffer methods (load_data, load_existing, etc.) can be directly invoked
* todo... continue

This notebook gives examples howto work with sc3nb.Buffers

In [ ]:
import numpy as np
import sc3nb as scn

In [ ]:
sc = scn.startup()

## Create Buffer from np Array

In [ ]:
d0 = np.random.rand(60000, 1)

In [ ]:
buf0 = sc.Buffer().load_data(d0[0:60000, 0])
buf0

In this case a default buffer with default sample rate (44100) and default insert mode (ToDo: '...') is created. If you want to create a buffer with a specific sample rate or OSC insertion method etc.. 

Attention: the OSC insertion is only possible for small datasets (less than 1000 entries).

**TODO: integrate looped insertion mode to cope with larger buffer insertions**

In [ ]:
buf0.info()  # not yet available
# how to see the lengths of the buffer?

In [ ]:
# uncomment following line to see help for the Buffer class:
# help(scn.Buffer)

In [ ]:
buf01 = sc.Buffer().load_data(d0[0:600, 0], sr=5, mode="osc")
buf01

## Create Buffer from PyA
Only theorical - I don't have the pya package to test it

Again, default transport method is mode='file', i.e. using a temporary file and fill the buffer on sc
with this content. 
* use mode="osc" to select the direct transfer of data via OSC messages (currently with size limits)

## Create Buffer of .wav File

In [ ]:
buf2 = sc.Buffer().load_file("./my_recording.wav")
buf2

In [ ]:
buf2.play()
print(buf2)
buf2.query()

The buffer method will automatically read the sample reate of the file and set it to Buffer.sr

## Allocate an empty Buffer

In [ ]:
buf3 = sc.Buffer().alloc(8*44100, sr=44100)
buf3

## Reuse an existing SC buffer
If you have already created a buffer in SC and want to have a sc3nb.Buffer object of that already existing buffer (identified by its bufnum), use Buffer.load_existing(bufnum):

In [ ]:
%sc b = Buffer.read(s, Platform.resourceDir +/+ "sounds/a11wlk01.wav");

In [ ]:
buf4num = %scg b.bufnum
buf4num
# ToDo: I got some odd values here such as '/b_allocRead' etc, on repeating finally I get the nr.
# check if this problem is gone with Dennis' improved cmd() interface

In [ ]:
buf4 = sc.Buffer().load_existing(buf4num)
buf4

## Copy an existing SC buffer
If you want to copy an already existing buffer into another buffer, you can do it:

In [ ]:
buf5 = sc.Buffer().copy_existing(buf4)

## Play Buffer

* The play method has the problem that different synths need to be used for different number of channels. 
    * The default play method creates the mono-channel synth 'pb-1ch'. 
    * For stereo buffers, manually overwrite by setting synth='pb-2ch'. 
* Future versions of Buffer should be able to determine the number of channels and automatically select an appropriate synth.

In [ ]:
%sc s.scope

In [ ]:
buf4.play()  # play at rate 1

In [ ]:
buf4.play(rate=2)  # play at given rate

In [ ]:
node_id = buf4.play(rate=4, loop=True)  # play looped 

In [ ]:
sc.msg("/n_free", node_id)  # free the buffer player 

## Write buffer content to file
Write the content of a buffer into a file. By default it is a .wav File with float as sample. You can change it via parameters "header" and "sample".

In [ ]:
buf4.write("./output.wav")

## Fill buffer with values

In [ ]:
buf3.fill(0, 8*44100, 700)

## Information about the buffer
Information about the buffer object:

In [ ]:
buf3

Information about the buffer in SC
(Known bug: You have to call this method multiple times, until you've got a list with the bufnum in the first list item)

In [ ]:
buf3.query()

## Delete/ Free buffer

* We start with a buffer

In [ ]:
buf2 = sc.Buffer().load_file("./my_recording.wav")
buf2.play(synth="pb-2ch")

In [ ]:
print(buf2)
buf2.query()

* now set all values of the buffer to zero:

In [ ]:
buf2.zero()

In [ ]:
print(buf2)
buf2.query()

Delete buffer in SC:

In [ ]:
buf2.free()

In [ ]:
print(buf2)
buf2.query()
# ToDo: Problem: after free, loaded is still true!!!